### Import Libraries

The first step is to import the libraries that we will need.

In [2]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

# for Big Query connections
# from google.cloud import bigquery
# from google.oauth2 import service_account
# import db_dtypes


In [3]:
wallaroo.__version__

'2023.2.1'

### Initialize connection

Start a connect to the Wallaroo instance and save the connection into the variable `wl`.

In [4]:
# Login through local Wallaroo instance

wl = wallaroo.Client()


wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

In [5]:
workspace_name = 'bikerental-nbz'
pipeline_name = 'bikeforecast-pipe'
model_name = 'bikeforecast-arima'

## Set the Workspace and Pipeline


In [6]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

# get a pipeline by name in the workspace
def get_pipeline(pname, create_if_absent=False):
    plist = wl.get_current_workspace().pipelines()
    pipeline = [p for p in plist if p.name() == pname]
    if len(pipeline) <= 0:
        if create_if_absent:
            pipeline = wl.build_pipeline(pname)
        else:
            raise KeyError(f"pipeline {pname} not found in this workspace")
    else:
        pipeline = pipeline[0]
    return pipeline

# Get the most recent version of a model in the workspace
# Assumes that the most recent version is the first in the list of versions.
# wl.get_current_workspace().models() returns a list of models in the current workspace

def get_model(mname):
    modellist = wl.get_current_workspace().models()
    model = [m.versions()[0] for m in modellist if m.name() == mname]
    if len(model) <= 0:
        raise KeyError(f"model {mname} not found in this workspace")
    return model[0]


workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)


{'name': 'bikerental-nbz', 'id': 18, 'archived': False, 'created_by': 'c3a45eb6-37ff-4020-8d59-7166c3e153d0', 'created_at': '2023-07-26T19:42:24.57956+00:00', 'models': [{'name': 'bikeforecast-arima', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 7, 26, 19, 42, 55, 544718, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 7, 26, 19, 42, 55, 544718, tzinfo=tzutc())}, {'name': 'bikeforecast-arima202', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 7, 26, 19, 47, 50, 853818, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 7, 26, 19, 47, 50, 853818, tzinfo=tzutc())}], 'pipelines': [{'name': 'bikeforecast-pipe', 'create_time': datetime.datetime(2023, 7, 26, 19, 42, 26, 35307, tzinfo=tzutc()), 'definition': '[]'}, {'name': 'bikeforecast-shadow', 'create_time': datetime.datetime(2023, 7, 26, 19, 47, 56, 429162, tzinfo=tzutc()), 'definition': '[]'}]}

### Fetch champion and upload challenger

Champion ARIMA(1,0,1) hard-coded to forecast seven days out.
Challenger ARIMA(2, 0, 2)


In [9]:
# champion model
champion = get_model(model_name)
display(champion)

challenger_name = 'bikeforecast-arima202'

challenger = get_model(challenger_name)
display(challenger)

Name,bikeforecast-arima
Version,d3ba8a8d-dcbf-477f-85e3-3670fdf81b85
File Name,forecast.py
SHA,7ed0600d8b754ff9d901633cf78de7f825c9d2d0f79a2647d271d2b04f4befd8
Status,ready
Image Path,None
Updated At,2023-26-Jul 19:42:55


Name,bikeforecast-arima202
Version,b5c9b362-a076-40c4-895e-49b99b963216
File Name,forecast-arma2.py
SHA,08424e36e41d5e2091c444bc45a74489a167ee420e24466e42a32d0bd4b18e47
Status,ready
Image Path,None
Updated At,2023-26-Jul 19:47:50


In [ ]:
# challenger_name = 'bikeforecast-arima202'
# challenger_file_name = './models/forecast-arma2.py'

# from wallaroo.framework import Framework

# challenger = wl.upload_model(challenger_name, challenger_file_name,  framework=Framework.PYTHON)

## Create a A/B Pipeline

In [10]:
abpipe_name = 'bikeforecast-ab'
pipeline = (wl.build_pipeline(abpipe_name)
            .add_random_split([(2, champion), (1, challenger)], "session_id"))

name,bikeforecast-ab
created,2023-07-26 19:54:22.191388+00:00
last_updated,2023-07-26 19:54:23.458690+00:00
deployed,True
tags,
versions,"67678963-7c20-4313-bba5-35bb50ada54c, f220fcfc-cd6f-457a-91ff-1d0910b1f523"
steps,bikeforecast-arima


In [28]:
# deploy
pipeline.deploy()

name,bikeforecast-ab
created,2023-07-26 19:54:22.191388+00:00
last_updated,2023-07-26 20:04:15.654831+00:00
deployed,True
tags,
versions,"ef1b1dcf-926b-4eb5-a945-b95429c695a8, 67678963-7c20-4313-bba5-35bb50ada54c, f220fcfc-cd6f-457a-91ff-1d0910b1f523"
steps,bikeforecast-arima


### Run Inference

Note that for a python runtime model,  the results are only from one model (the champion).


In [11]:
# show the input data
pd.read_json('./data/testdata_dict.json')


,dteday,site_id,cnt,season,holiday,weekday,workingday
0,2011-02-02,site0001,1240,1,0,3,1
1,2011-02-03,site0001,1551,1,0,4,1
2,2011-02-04,site0001,2324,1,0,5,1
3,2011-02-05,site0001,805,1,0,6,0
4,2011-02-06,site0001,1948,1,0,0,0
5,2011-02-07,site0001,1650,1,0,1,1
6,2011-02-08,site0001,913,1,0,2,1
7,2011-02-09,site0001,931,1,0,3,1
8,2011-02-10,site0001,1256,1,0,4,1
9,2011-02-11,site0001,1614,1,0,5,1


In [42]:
# simple test

for x in range(10):
    inferencedata = json.load(open("./data/testdata_dict.json"))
    results = pipeline.infer(inferencedata)
    # results = pipeline.infer_from_file('./data/testdata_dict.json', data_format="custom-json")
    results

In [30]:
resultframe = pd.DataFrame(results[0])
resultframe

,dteday,site_id,forecast
0,2011-03-02,site0001,2269
1,2011-03-03,site0001,1712
2,2011-03-04,site0001,1795
3,2011-03-05,site0001,1371
4,2011-03-06,site0001,1819
5,2011-03-07,site0001,2045
6,2011-03-08,site0001,1974


To see the forecasts from both the champion and the challenger, you must look at the logs.

In [53]:
logs = pipeline.logs(dataset=["time", "out.json","metadata"])
display(logs)

,time,out.json,metadata.last_model,metadata.pipeline_version,metadata.elapsed,metadata.dropped
0,2023-07-26 20:09:28.628,"{""dteday"":{""0"":""2011-03-02"",""1"":""2011-03-03"",""2"":""2011-03-04"",""3"":""2011-03-05"",""4"":""2011-03-06"",""5"":""2011-03-07"",""6"":""2011-03-08""},""site_id"":{""0"":""site0001"",""1"":""site0001"",""2"":""site0001"",""3"":""site0001"",""4"":""site0001"",""5"":""site0001"",""6"":""site0001""},""forecast"":{""0"":1880,""1"":1454,""2"":1721,""3"":1385,""4"":1617,""5"":1706,""6"":1725}}","{""model_name"":""bikeforecast-arima202"",""model_sha"":""08424e36e41d5e2091c444bc45a74489a167ee420e24466e42a32d0bd4b18e47""}",ef1b1dcf-926b-4eb5-a945-b95429c695a8,"[44201, 208789311]",[]
1,2023-07-26 20:09:28.138,"{""dteday"":{""0"":""2011-03-02"",""1"":""2011-03-03"",""2"":""2011-03-04"",""3"":""2011-03-05"",""4"":""2011-03-06"",""5"":""2011-03-07"",""6"":""2011-03-08""},""site_id"":{""0"":""site0001"",""1"":""site0001"",""2"":""site0001"",""3"":""site0001"",""4"":""site0001"",""5"":""site0001"",""6"":""site0001""},""forecast"":{""0"":2269,""1"":1712,""2"":1795,""3"":1371,""4"":1819,""5"":2045,""6"":1974}}","{""model_name"":""bikeforecast-arima"",""model_sha"":""7ed0600d8b754ff9d901633cf78de7f825c9d2d0f79a2647d271d2b04f4befd8""}",ef1b1dcf-926b-4eb5-a945-b95429c695a8,"[49601, 116505101]",[]
2,2023-07-26 20:09:27.606,"{""dteday"":{""0"":""2011-03-02"",""1"":""2011-03-03"",""2"":""2011-03-04"",""3"":""2011-03-05"",""4"":""2011-03-06"",""5"":""2011-03-07"",""6"":""2011-03-08""},""site_id"":{""0"":""site0001"",""1"":""site0001"",""2"":""site0001"",""3"":""site0001"",""4"":""site0001"",""5"":""site0001"",""6"":""site0001""},""forecast"":{""0"":2269,""1"":1712,""2"":1795,""3"":1371,""4"":1819,""5"":2045,""6"":1974}}","{""model_name"":""bikeforecast-arima"",""model_sha"":""7ed0600d8b754ff9d901633cf78de7f825c9d2d0f79a2647d271d2b04f4befd8""}",ef1b1dcf-926b-4eb5-a945-b95429c695a8,"[49400, 118215022]",[]
3,2023-07-26 20:09:27.073,"{""dteday"":{""0"":""2011-03-02"",""1"":""2011-03-03"",""2"":""2011-03-04"",""3"":""2011-03-05"",""4"":""2011-03-06"",""5"":""2011-03-07"",""6"":""2011-03-08""},""site_id"":{""0"":""site0001"",""1"":""site0001"",""2"":""site0001"",""3"":""site0001"",""4"":""site0001"",""5"":""site0001"",""6"":""site0001""},""forecast"":{""0"":2269,""1"":1712,""2"":1795,""3"":1371,""4"":1819,""5"":2045,""6"":1974}}","{""model_name"":""bikeforecast-arima"",""model_sha"":""7ed0600d8b754ff9d901633cf78de7f825c9d2d0f79a2647d271d2b04f4befd8""}",ef1b1dcf-926b-4eb5-a945-b95429c695a8,"[45801, 116856105]",[]
4,2023-07-26 20:09:26.560,"{""dteday"":{""0"":""2011-03-02"",""1"":""2011-03-03"",""2"":""2011-03-04"",""3"":""2011-03-05"",""4"":""2011-03-06"",""5"":""2011-03-07"",""6"":""2011-03-08""},""site_id"":{""0"":""site0001"",""1"":""site0001"",""2"":""site0001"",""3"":""site0001"",""4"":""site0001"",""5"":""site0001"",""6"":""site0001""},""forecast"":{""0"":2269,""1"":1712,""2"":1795,""3"":1371,""4"":1819,""5"":2045,""6"":1974}}","{""model_name"":""bikeforecast-arima"",""model_sha"":""7ed0600d8b754ff9d901633cf78de7f825c9d2d0f79a2647d271d2b04f4befd8""}",ef1b1dcf-926b-4eb5-a945-b95429c695a8,"[53501, 122873278]",[]
5,2023-07-26 20:09:25.963,"{""dteday"":{""0"":""2011-03-02"",""1"":""2011-03-03"",""2"":""2011-03-04"",""3"":""2011-03-05"",""4"":""2011-03-06"",""5"":""2011-03-07"",""6"":""2011-03-08""},""site_id"":{""0"":""site0001"",""1"":""site0001"",""2"":""site0001"",""3"":""site0001"",""4"":""site0001"",""5"":""site0001"",""6"":""site0001""},""forecast"":{""0"":1880,""1"":1454,""2"":1721,""3"":1385,""4"":1617,""5"":1706,""6"":1725}}","{""model_name"":""bikeforecast-arima202"",""model_sha"":""08424e36e41d5e2091c444bc45a74489a167ee420e24466e42a32d0bd4b18e47""}",ef1b1dcf-926b-4eb5-a945-b95429c695a8,"[66001, 215991739]",[]
6,2023-07-26 20:09:25.478,"{""dteday"":{""0"":""2011-03-02"",""1"":""2011-03-03"",""2"":""2011-03-04"",""3"":""2011-03-05"",""4"":""2011-03-06"",""5"":""2011-03-07"",""6"":""2011-03-08""},""site_id"":{""0"":""

In [26]:
logs.columns

Index(['time', 'in.json', 'out.json', 'check_failures'], dtype='object')

In [ ]:
# convenience function to retrieve output from custom json inference result 
# assume that the result is a dictionary representation of a data frame
def get_output(inf_result):
    rdict = inf_result.raw['outputs'][0]['Json']['data'][0]
    return pd.DataFrame.from_dict(rdict)

In [ ]:
champion = logs['out.json'][0]
pd.read_json(champion)

In [ ]:
challenger = logs['out_bikeforecast-arima202.json'][0]
pd.read_json(challenger)

### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [ ]:
pipeline.undeploy()